In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
import MetaTrader5 as mt5
# from vectorbt.portfolio import Portfolio as vbt_Portfolio

# Connection

In [ ]:
# RICO_REAL   = { 'login':16331371,   'password':'#000000Zz', 'server':'Rico-PRD'                 }
# CLEAR_REAL  = { 'login':1018156111, 'password':'#000000Zz', 'server':'ClearInvestimentos-CLEAR' }

# PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

# assert mt5.initialize(PATH_7, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H3','D1']

# Load

In [ ]:
# def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
#     Pipe = []
#     for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
#         timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
#         Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
#         Raw.insert(0, 'a',          Raw.index + 1)
#         Raw.insert(1, 'z',          Raw.index - Raw.index.max())
#         Raw.insert(2, 'symbol',     symbol)
#         Raw.insert(3, 'tf',         TF)
#         Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
#         Pipe.append(Raw)
#     pass
#     return pd.concat(Pipe, ignore_index=1)

# Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
# Raw.head()

In [ ]:
# Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    Src['Week.Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Change Abs'] = Src['Change'].abs()
    Calc['Change Pos'] = Src['Change'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Change Neg'] = Src['Change'].apply(lambda x: x if (x <  0) else nan)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [102]:
Calc.groupby('Week.Day')[['HL']].describe().round(0).astype(int)

HL                                         
         count  mean   std  min   25%   50%   75%   max
Week.Day                                               
2          247  2414  1168  593  1557  2220  3004  8487
3          250  2503  1119  655  1668  2208  3268  7864
4          252  2565  1126  687  1806  2398  3051  7753
5          249  2653  1233  701  1713  2454  3197  7278
6          247  2601  1118  805  1808  2411  3227  7479

In [103]:
Calc.groupby('Week.Day')[['Change']].describe().round(0).astype(int)

Change                                        
          count mean   std   min   25%  50%   75%   max
Week.Day                                               
2           247    0  1711 -7641  -993   87  1078  5768
3           250   11  1641 -4544 -1024   64  1015  5218
4           252  -27  1631 -4825 -1009  102   938  4619
5           249   -8  1865 -5935 -1169   80  1118  5442
6           247  -72  1703 -5367 -1220  -76   962  5178

In [104]:
Calc.groupby('Week.Day')[['Change Abs']].describe().round(0).astype(int)

Change Abs                                       
              count  mean   std min  25%   50%   75%   max
Week.Day                                                  
2               247  1278  1135   6  477  1054  1687  7641
3               250  1276  1029   0  512  1022  1712  5218
4               252  1272  1018   7  487   959  1831  4825
5               249  1453  1166   0  524  1151  2095  5935
6               247  1352  1034   6  524  1064  1968  5367

In [105]:
Calc.groupby('Week.Day')[['Change Pos']].describe().round(0).astype(int)

Change Pos                                       
              count  mean   std min  25%   50%   75%   max
Week.Day                                                  
2               134  1178   976   6  450   986  1697  5768
3               131  1228  1011   0  362   983  1728  5218
4               129  1216   969  13  452   890  1892  4619
5               131  1373  1132   0  492  1046  1976  5442
6               120  1317  1015   6  504   974  1930  5178

In [106]:
Calc.groupby('Week.Day')[['Change Neg']].describe().round(0).astype(int)

Change Neg                                       
              count  mean   std   min   25%   50%  75% max
Week.Day                                                  
2               113 -1396  1293 -7641 -1677 -1083 -606 -16
3               119 -1329  1050 -4544 -1680 -1074 -594  -6
4               123 -1331  1068 -4825 -1757 -1036 -499  -7
5               118 -1541  1200 -5935 -2119 -1224 -595 -13
6               127 -1385  1055 -5367 -1975 -1181 -584 -11